In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14455922387779860214, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1147236115826743640
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4115333576258674040
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11281491559
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 869486837589980795
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
!pip install pydrive

    100% |████████████████████████████████| 993kB 22.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np
import pandas as pd
import h5py
import sklearn
import keras
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

Using TensorFlow backend.


In [0]:
from sklearn.metrics import classification_report, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, load_model
from keras.layers import Dense, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.regularizers import l2, l1

In [0]:
from keras import backend as K
K.set_image_dim_ordering('th')

In [0]:
img_width, img_height = 256, 256
train_data_dir = '/content/drive/My Drive/Damaged Car Validation/training'
validation_data_dir = '/content/drive/My Drive/Damaged Car Validation/validation'

In [0]:
def initialize_layers():
    model = Sequential()
    
    model.add(ZeroPadding2D((1,1), input_shape = (3, img_width, img_height)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(ZeroPadding2D((1,1), input_shape = (3, img_width, img_height)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(ZeroPadding2D((1,1), input_shape = (3, img_width, img_height)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(ZeroPadding2D((1,1), input_shape = (3, img_width, img_height)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    return model

In [0]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40,
                                   width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_height, img_width),
                                                   batch_size=8, class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                           target_size=(img_height, img_width),
                                                           batch_size=8, class_mode='binary')

model = initialize_layers()
model.add(Flatten())
model.add(Dense(256, activation = 'relu', W_regularizer=l2(0.01)))
model.add(Dropout(0.5)) 
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer = optimizers.SGD(lr=0.001, momentum=0.9),
              metrics=['accuracy'])

model.fit_generator(train_generator, samples_per_epoch = 1915,
                    nb_epoch = 75, validation_data=validation_generator,
                    nb_val_samples = 460)

Found 1915 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update yo

Epoch 1/75
 37/239 [===>..........................] - ETA: 7:03 - loss: 5.7115 - acc: 0.5169

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


239/239 [==============================] - 618s 3s/step - loss: 5.5182 - acc: 0.5136 - val_loss: 5.2910 - val_acc: 0.5000
Epoch 2/75
239/239 [==============================] - 87s 363ms/step - loss: 5.0767 - acc: 0.5192 - val_loss: 4.8739 - val_acc: 0.4992
Epoch 3/75
239/239 [==============================] - 86s 361ms/step - loss: 4.6757 - acc: 0.5206 - val_loss: 4.4883 - val_acc: 0.4973
Epoch 4/75
239/239 [==============================] - 86s 361ms/step - loss: 4.3103 - acc: 0.5342 - val_loss: 4.1515 - val_acc: 0.5014
Epoch 5/75
239/239 [==============================] - 85s 357ms/step - loss: 3.9791 - acc: 0.5347 - val_loss: 3.8219 - val_acc: 0.5014
Epoch 6/75
239/239 [==============================] - 85s 357ms/step - loss: 3.6617 - acc: 0.5778 - val_loss: 3.5204 - val_acc: 0.6083
Epoch 7/75
239/239 [==============================] - 87s 364ms/step - loss: 3.3906 - acc: 0.6013 - val_loss: 3.2593 - val_acc: 0.5743
Epoch 8/75
239/239 [==============================] - 86s 361ms/step

In [0]:
model.save('car_damage_validation.h5')

In [0]:
model_json = model.to_json()
with open("/content/drive/My Drive/Damaged Car Validation/car_damage_validation.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/My Drive/Damaged Car Validation/car_damage_validation.h5")

In [0]:
model.save('/content/drive/My Drive/Damaged Car Validation/car_damage_validation.h5')

In [0]:
import pickle

In [0]:
with open('/content/drive/My Drive/Damaged Car Validation/car_damage_validation.pickle', 'wb') as mod:
  pickle.dump(model, mod)